In [ ]:
import tensorflow as tf
import cv2
import numpy as np
from tensorflow import keras
from pathlib import Path
from natsort import natsorted

In [ ]:
batch_size=10
patch_size=16
img_size=128
num_patches=(img_size//patch_size)**2
p_dim=256

In [ ]:
target_class={0:"Normal",1:"Tuberculosis"}

In [ ]:
Normal_dir=Path("TB_Chest_Radiography_Database/Normal/")
TB_dir=Path("TB_Chest_Radiography_Database/Tuberculosis/")

In [ ]:
Normal_images:list=natsorted(list(map(str, list(Normal_dir.glob("*.png")))))
TB_images:list=natsorted(list(map(str, list(TB_dir.glob("*.png")))))

In [ ]:
Normal_labels:list=[0]*len(Normal_images)
TB_labels:list=[1]*len(TB_images)

In [ ]:
images=np.array(Normal_images+TB_images)
labels=np.array(Normal_labels+TB_labels)
images.shape, labels.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(images,labels,test_size=0.2,random_state=42)
x_valid,x_test,y_valid,y_test=train_test_split(x_valid,y_valid,test_size=0.5,random_state=42)
(x_train.shape,x_valid.shape,x_test.shape)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
image_1=x_train[0]
image_1=cv2.imread(image_1)
image_1=cv2.cvtColor(image_1,cv2.COLOR_BGR2GRAY)
clahe=cv2.createCLAHE(2,(16,16))
image_1_equalized=clahe.apply(image_1)
image_1_equalized_blurred=cv2.GaussianBlur(image_1_equalized,(5,5),0)
_, ax = plt.subplots(1, 2, figsize=(7, 7))
ax[0].imshow(image_1,cmap='gray')
ax[0].set_title("Before CLAHE + Gaussian Blur")
ax[0].axis("off")
ax[1].imshow(image_1_equalized_blurred,cmap='gray')
ax[1].set_title("After CLAHE + Gaussian Blur")
ax[1].axis("off")

In [ ]:
def image_preprocessing(path):
    img=cv2.imread(path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    clahe=cv2.createCLAHE(3,(16,16))
    img=clahe.apply(img)
    img=cv2.GaussianBlur(img,(5,5),0)
    img=cv2.resize(img,(img_size,img_size))
    img=img.reshape(img_size,img_size,1)
    return img

In [ ]:
#import albumentations as alb

In [ ]:
#augmentation=alb.Compose([
    #alb.Rotate(limit=5,interpolation=cv2.INTER_LINEAR,p=1),
    #alb.Affine(translate_percent=0.05,interpolation=cv2.INTER_LINEAR,p=1),
#])

In [ ]:
#for path in x_train[y_train==1]:
    #img=cv2.imread(path)
    #try:
        #augmented=augmentation(image=img)
        #modified_path=path.split("\\")[2]
        #cv2.imwrite(f'augmented_images/{modified_path.split(".")[0]}.png',augmented['image'])
    #except Exception as e:
        #print(e)

In [ ]:
augmented_dir=Path("augmented_images")

In [ ]:
augmented_images=list(map(str, list(augmented_dir.glob("*.png"))))
augmented_labels=[1]*len(augmented_images)
augmented_images=np.array(augmented_images)
augmented_labels=np.array(augmented_labels)

In [ ]:
x_train=np.concatenate((x_train,augmented_images))
y_train=np.concatenate((y_train,augmented_labels))
x_train.shape,y_train.shape

In [ ]:
x_train=np.array(list(map(image_preprocessing,x_train)))
x_train.shape

In [ ]:
x_valid=np.array(list(map(image_preprocessing,x_valid)))
x_valid.shape

In [ ]:
x_test=np.array(list(map(image_preprocessing,x_test)))
x_test.shape

In [ ]:
train_dataset=tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(4000,seed=42).batch(batch_size).prefetch(tf.data.AUTOTUNE)
validation_dataset=tf.data.Dataset.from_tensor_slices((x_valid,y_valid)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset=tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [ ]:
_, ax = plt.subplots(1, 10, figsize=(14, 4))
for batch in train_dataset.take(1):
    images = batch[0]
    labels = batch[1]
    for i in range(batch_size):
        img = (images[i]*255).numpy().astype("uint8")
        label=labels[i].numpy()
        ax[i].imshow(img,cmap='gray')
        ax[i].set_title(target_class[label])
        ax[i].axis("off")

In [ ]:
_, ax = plt.subplots(1, 10, figsize=(14, 4))
for batch in validation_dataset.take(1):
    images = batch[0]
    labels = batch[1]
    for i in range(batch_size):
        img = (images[i]*255).numpy().astype("uint8")
        label=labels[i].numpy()
        ax[i].imshow(img,cmap='gray')
        ax[i].set_title(target_class[label])
        ax[i].axis("off")

In [ ]:
_, ax = plt.subplots(1, 10, figsize=(14, 4))
for batch in test_dataset.take(1):
    images = batch[0]
    labels = batch[1]
    for i in range(batch_size):
        img = (images[i]*255).numpy().astype("uint8")
        label=labels[i].numpy()
        ax[i].imshow(img,cmap='gray')
        ax[i].set_title(target_class[label])
        ax[i].axis("off")

In [ ]:
class Patches(keras.layers.Layer):
    def __init__(self, patch_size):
        super().__init__()
        self.patch_size = patch_size
    
    def get_config(self):
        config=super().get_config()
        config.update({
            "patch_size":self.patch_size
        })
        return config

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(4, 4))
image = x_valid[np.random.choice(range(x_valid.shape[0]))]
plt.imshow((image*255).astype("uint8"),cmap='gray')
plt.axis("off")

image=tf.convert_to_tensor([image])

patches = Patches(patch_size)(image)
print(f"Image size: {img_size} X {img_size}")
print(f"Patch size: {patch_size} X {patch_size}")
print(f"Patches per image: {patches.shape[1]}")
print(f"Elements per patch: {patches.shape[-1]}")

n = int(np.sqrt(patches.shape[1]))
plt.figure(figsize=(4, 4))
for i, patch in enumerate(patches[0]):
    ax = plt.subplot(n, n, i + 1)
    patch_img = tf.reshape(patch, (patch_size, patch_size, 1))
    plt.imshow((patch_img*255).numpy().astype("uint8"),cmap='gray')
    plt.axis("off")

In [ ]:
class PatchEncoder(keras.layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super().__init__()
        self.num_patches = num_patches
        self.projection_dim=projection_dim
        self.projection = keras.layers.Dense(units=projection_dim)
        self.position_embedding = keras.layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )
    
    def get_config(self):
        config=super().get_config()
        config.update({
            "num_patches":self.num_patches,
            "projection_dim":self.projection_dim
        })
        return config

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

In [ ]:
def mlp(x, hidden_units):
    for units in hidden_units:
        x = keras.layers.Dense(units, activation=tf.nn.gelu)(x)
    return x

In [ ]:
import keras_tuner as kt

In [ ]:
def build_model(hp):
    inputs=keras.layers.Input(shape=(img_size,img_size,1))
    patches=Patches(patch_size)(inputs)
    encoded=PatchEncoder(num_patches,p_dim)(patches)
    heads_att=hp.Int("n_self_heads",min_value=2,max_value=6,step=2)
    num_encoder=hp.Int("n_encoders",min_value=6,max_value=12,step=3)
    for _ in range(num_encoder):
        x1=keras.layers.LayerNormalization()(encoded)
        attention=keras.layers.MultiHeadAttention(num_heads=heads_att,key_dim=p_dim)(x1,x1)
        x2=keras.layers.Add()([attention,encoded])
        x3=keras.layers.LayerNormalization()(x2)
        x3=mlp(x3,[2*p_dim,p_dim])
        encoded=keras.layers.Add()([x3,x2])
    rep=keras.layers.GlobalAveragePooling1D()(encoded)
    mlp_head=hp.Int("n_mlp_head",min_value=1024,max_value=3072,step=1024)
    rep=mlp(rep,hidden_units=[mlp_head,mlp_head//2])
    output=keras.layers.Dense(1,activation='sigmoid')(rep)

    model=keras.Model(inputs=inputs,outputs=output)
    opt = keras.optimizers.Adam()
    model.compile(optimizer=opt,
                  loss=keras.losses.BinaryFocalCrossentropy(),
                  metrics=[keras.metrics.Recall(),
                           keras.metrics.BinaryAccuracy()])
    return model

In [ ]:
tuner=kt.GridSearch(build_model,objective=kt.Objective('val_loss','min'))
tuner.search_space_summary()

In [ ]:
early_stop=keras.callbacks.EarlyStopping(monitor='val_loss',restore_best_weights=False,patience=10)
lr_scheduler=keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=3,verbose=1,min_lr=1e-5)

In [ ]:
tuner.search(train_dataset,
             batch_size=batch_size,
             epochs=50,
             validation_data=validation_dataset,
             callbacks=[early_stop,lr_scheduler])
model=tuner.get_best_models()[0]

In [ ]:
tuner.results_summary(3)

In [ ]:
model.summary()

In [ ]:
#keras.utils.plot_model(model,show_layer_activations=True,show_shapes=True)

In [ ]:
early_stop=keras.callbacks.EarlyStopping(monitor='val_loss',restore_best_weights=True,patience=10)
lr_scheduler=keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=3,verbose=1,min_lr=1e-5)

In [ ]:
history=model.fit(train_dataset,
                  validation_data=validation_dataset,
                  epochs=100,
                  callbacks=[early_stop,lr_scheduler])

In [ ]:
_, ax = plt.subplots(1, 3, figsize=(20,7))
loss = history.history['loss']
val_loss = history.history['val_loss']
recall=history.history['recall']
val_recall=history.history['val_recall']
binary_accuracy=history.history['binary_accuracy']
val_binary_accuracy=history.history['val_binary_accuracy']
epochs = range(len(loss))

ax[0].plot(epochs, loss)
ax[0].plot(epochs, val_loss)
ax[0].legend(['loss', 'val_loss'], loc='upper right')

ax[1].plot(epochs, recall)
ax[1].plot(epochs, val_recall)
ax[1].legend(['recall', 'val_recall'], loc='upper right')

ax[2].plot(epochs, binary_accuracy)
ax[2].plot(epochs, val_binary_accuracy)
ax[2].legend(['binary_accuracy', 'val_binary_accuracy'], loc='upper right')

In [ ]:
y_pred_val=model.predict(validation_dataset)
y_pred_val=np.round(y_pred_val)

In [ ]:
y_pred_test=model.predict(test_dataset)
y_pred_test=np.round(y_pred_test)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report


def evaluation_parametrics(name,y_val, y_pred):
    
    print("\n------------------------{}------------------------\n".format(name))

    cm_test = confusion_matrix(y_val, y_pred)
    t1 = ConfusionMatrixDisplay(cm_test)    
    print("\nClassification Report for Data Validation\n")
    print(classification_report(y_val, y_pred))   
    print("--------------------------------------------------------------------------")

    t1.plot()

In [ ]:
evaluation_parametrics("Machine Learning - Classification", y_valid, y_pred_val)

In [ ]:
evaluation_parametrics("Machine Learning - Classification", y_test, y_pred_test)

In [ ]:
model.save('ViT_TBC_CXR.h5')